In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
browser = Browser('chrome')

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card"
browser.visit(url)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

In [3]:
# To select "English" as preffered language
results = browser.links.find_by_text('English')
results.click()

In [4]:
# This is for loading more products 
max_iteration = 1
iteration = 0

while iteration<= max_iteration:
    # Extract the HTML of the current page
    html = browser.html
    
    # Create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')
    
    all_links = soup.find_all("div", class_ = 'coveo-product-items')

    links = []
    
    for link in all_links:
        results = link.a['href']
        links.append(results)
        
    # Find the element by ID
    element = soup.find(id='loadMore')
    
    if element is None:
        # Element not found, exit the loop
        break
        
    # Wait for 2 seconds
    time.sleep(1)
    
    
        
    # Click on the element
    browser.find_by_id('loadMore').click()
    
    
    
    iteration += 1

In [5]:
print(links)

['https://www.lcbo.com/en/collavini-pinot-grigio-33340', 'https://www.lcbo.com/en/topo-chico-hard-seltzer-signature-margarita-31234', 'https://www.lcbo.com/en/josh-cellars-pinot-grigio-15754', 'https://www.lcbo.com/en/ruffino-prosecco-rose-doc-20896', 'https://www.lcbo.com/en/woodbridge-by-robert-mondavi-cabernet-sauvignon-48611', 'https://www.lcbo.com/en/kim-crawford-sauvignon-blanc-35386', 'https://www.lcbo.com/en/tom-gore-cabernet-sauvignon-451336', 'https://www.lcbo.com/en/palm-bay-paradise-twist-24333', 'https://www.lcbo.com/en/nude-pink-lemonade-24369', 'https://www.lcbo.com/en/bartenders-trading-pina-colada-31238', 'https://www.lcbo.com/en/meukow-cognac-vs-382580', 'https://www.lcbo.com/en/aperol-spritz-ready-to-serve-24345', 'https://www.lcbo.com/en/seventh-heaven-classic-london-dry-gin-30719', 'https://www.lcbo.com/en/salvadors-grapefruit-paloma-31239', 'https://www.lcbo.com/en/herradura-reposado-tequila-452615', 'https://www.lcbo.com/en/kim-crawford-ros-c-650325', 'https://ww